In [9]:
import pandas as pd

# load data
df = pd.read_csv("rq2_results_relative_hr.csv")

# --- Constructiveness computations ---

# Both promote (pp)
df["CT_A_pp"] = df["rA_pp"] - df["rA_prom"]   # effect of B on A
df["CT_B_pp"] = df["rB_pp"] - df["rB_prom"]   # effect of A on B

# Both demote (dd)
df["CT_A_dd"] = df["rA_dd"] - df["rA_demo"]   # effect of B on A
df["CT_B_dd"] = df["rB_dd"] - df["rB_demo"]   # effect of A on B

# A promotes, B demotes (pd)
df["CT_A_pd"] = df["rA_pd"] - df["rA_prom"]   # effect of B on A (A=promoter)
df["CT_B_pd"] = df["rB_pd"] - df["rB_demo"]   # effect of A on B (B=demoter)

# A demotes, B promotes (dp)
df["CT_A_dp"] = df["rA_dp"] - df["rA_demo"]   # effect of B on A (A=demoter)
df["CT_B_dp"] = df["rB_dp"] - df["rB_prom"]   # effect of A on B (B=promoter)

# --- Averages for reporting ---
metrics_cols = [
    # relative HR
    "rA_prom","rB_prom","rA_demo","rB_demo",
    "rA_pp","rB_pp","rA_dd","rB_dd",
    "rA_pd","rB_pd","rA_dp","rB_dp",
    # constructiveness (both directions)
    "CT_A_pp","CT_B_pp","CT_A_dd","CT_B_dd",
    "CT_A_pd","CT_B_pd","CT_A_dp","CT_B_dp"
]

avg_results = df[metrics_cols].mean().reset_index()
avg_results.columns = ["metric", "mean"]

# save
avg_results.to_csv("rq2_results_recomputed_ct.csv", index=False)

print(avg_results)



     metric      mean
0   rA_prom  1.056916
1   rB_prom  1.154793
2   rA_demo  1.391383
3   rB_demo  0.686838
4     rA_pp  0.787302
5     rB_pp  2.248927
6     rA_dd  2.029252
7     rB_dd  0.673820
8     rA_pd  0.622449
9     rB_pd  1.000000
10    rA_dp  0.862132
11    rB_dp  1.483548
12  CT_A_pp -0.269615
13  CT_B_pp  1.094134
14  CT_A_dd  0.637868
15  CT_B_dd -0.013019
16  CT_A_pd -0.434467
17  CT_B_pd  0.313162
18  CT_A_dp -0.529252
19  CT_B_dp  0.328755


In [3]:
df

,N,p,trial,gA_base,gB_base,gA_prom,gA_demo,gB_prom,gB_demo,gA_pp,...,rA_pp,rB_pp,rA_dd,rB_dd,rA_pd,rB_pd,rA_dp,rB_dp,size_A,size_B
0,10,0.1,0,0.050000,0.145530,0.00,0.00,0.00,0.0,0.20,...,4.000000,0.000000,0.000000,0.000000,0.000000,2.061433,0.000000,0.000000,10,10
1,10,0.1,1,0.046523,0.152152,0.00,0.00,0.00,0.2,0.00,...,0.000000,0.000000,0.000000,1.971708,0.000000,1.314472,0.000000,0.000000,10,10
2,10,0.1,2,0.051490,0.145530,0.00,0.10,0.00,0.1,0.00,...,0.000000,0.000000,0.000000,1.374289,0.000000,0.000000,0.000000,0.000000,10,10
3,10,0.1,3,0.047185,0.155795,0.00,0.00,0.00,0.2,0.00,...,0.000000,0.000000,0.000000,1.283741,0.000000,2.567481,2.119298,0.000000,10,10
4,10,0.1,4,0.048344,0.152318,0.00,0.00,0.00,0.0,0.00,...,0.000000,0.000000,0.000000,0.656522,0.000000,1.969565,0.000000,0.000000,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,50,1.0,5,0.018874,0.001821,0.52,0.04,0.06,0.0,0.02,...,1.059649,32.945455,0.000000,0.000000,1.059649,0.000000,0.000000,318.472727,50,50
146,50,1.0,6,0.018709,0.001159,0.86,0.06,0.08,0.0,0.54,...,28.863717,379.657143,0.000000,0.000000,22.449558,17.257143,1.069027,465.942857,50,50
147,50,1.0,7,0.017881,0.002152,0.14,0.00,0.14,0.0,0.22,...,12.303704,278.769231,0.000000,0.000000,30.200000,0.000000,0.000000,92.923077,50,50
148,50,1.0,8,0.015563,0.001656,0.06,0.00,0.22,0.0,0.22,...,14.136170,265.760000,1.285106,0.000000,51.404255,0.000000,1.285106,108.720000,50,50
